# Given object mesh, create a camera trajectory and render simulated camera images

In [ ]:
import pyrender
import trimesh
import numpy as np
import os, sys
import os.path as osp
import matplotlib.pyplot as plt
from utils import *
from trimesh.visual import texture, TextureVisuals
from PIL import Image
from trimesh import Trimesh

In [ ]:
obj_name = "strawberry"

shape_mapping_root_path = osp.normpath(osp.join(osp.abspath(''), '..', '..', '..'))
print(f"shape_mapping_root_path: {shape_mapping_root_path}")

# manually copy the google_512k folder from model/xxx_obj_name to processed_data/obj_name
obj_mesh_path = osp.join(shape_mapping_root_path, "processed_data", obj_name, "google_512k", "textured.obj")


# Load object mesh using trimesh
obj_mesh = trimesh.load(obj_mesh_path)

# set the mesh to red color
obj_mesh.visual.face_colors = [255, 0, 0, 255]
obj_mesh.visual.vertex_colors = [255, 0, 0, 255]
# obj_mesh.show()

# save the mesh to a file
obj_mesh.export(osp.join(shape_mapping_root_path, "processed_data", obj_name, "google_512k", "textured_red.obj"))

# # add texture map
# texture_image = Image.open(osp.join(shape_mapping_root_path, "processed_data", obj_name, "google_512k", "texture_map.png"))
# texture = trimesh.visual.texture.TextureVisuals(
#     uv=(obj_mesh.vertices[:, :2] - np.min(obj_mesh.vertices[:, :2], axis=0)) / np.ptp(obj_mesh.vertices[:, :2], axis=0),
#     image=texture_image
# )
# obj_mesh.visual = texture

In [ ]:
def create_camera_pose(azimuth, elevation, distance, target= [0, 0, 0]):
    """
    Create a camera pose matrix given azimuth, elevation, and distance from the origin.
    The camera is looking at the origin.
    :param azimuth: Azimuth angle in degrees
    :param elevation: Elevation angle in degrees
    :param distance: Distance from the origin
    :return: 4x4 camera pose matrix
    """
    # Convert azimuth and elevation to radians
    azimuth_rad = np.radians(azimuth)
    elevation_rad = np.radians(elevation)

    # Calculate camera position in 3D space
    x = distance * np.cos(elevation_rad) * np.cos(azimuth_rad)
    y = distance * np.cos(elevation_rad) * np.sin(azimuth_rad)
    z = distance * np.sin(elevation_rad)
    print(f"azimuth: {azimuth}, elevation: {elevation}, distance: {distance}, x: {x}, y: {y}, z: {z}")
    # Define camera position vector
    camera_position = np.array([x, y, z])

    # Define camera target point (looking at origin)
    camera_target = np.array(target)

    # Define up vector (pointing upwards in the world frame)
    up_vector = np.array([0, 0, 1])

    # Calculate camera z-axis (direction camera is pointing)
    camera_z_axis = camera_target - camera_position
    camera_z_axis /= np.linalg.norm(camera_z_axis)
    # reverse the z-axis so that +z is pointing inward (follow pyrender's convention)
    camera_z_axis = -camera_z_axis

    # Calculate camera x-axis (right vector)
    camera_x_axis = np.cross(up_vector, camera_z_axis)
    camera_x_axis /= np.linalg.norm(camera_x_axis)

    # Calculate camera y-axis (up vector)
    camera_y_axis = np.cross(camera_z_axis, camera_x_axis)
    camera_y_axis /= np.linalg.norm(camera_y_axis)

    # Create camera rotation matrix
    rotation_matrix = np.column_stack((camera_x_axis, camera_y_axis, camera_z_axis))

    # Combine rotation and translation to get camera pose matrix
    camera_pose = np.eye(4)
    camera_pose[:3, :3] = rotation_matrix
    camera_pose[:3, 3] = camera_position
    return camera_pose



In [ ]:
# Set up the camera pose
# create camera pose based on azimuth, elevation, distance

# Example usage:
azimuth = 45  # degrees
elevation = 45  # degrees
distance = 0.1  # distance from camera to target
target = [0, 0.0, 0.0]  # target point (where the camera is looking at)
camera_pose = create_camera_pose(azimuth, elevation, distance)

# # Sanity check
# # Visualize the camera pose in the 3D plot
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# # Plot camera position
# ax.scatter(camera_pose[0, 3], camera_pose[1, 3], camera_pose[2, 3], c='r', marker='o')
# ax.scatter(0, 0, 0, c='g', marker='o')
# # add legend
# ax.legend(['Camera', 'Origin'])
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# set_axes_equal(ax)
# plt.show()

# # Visualize the camera and object mesh in a trimesh scene
# # Create a trimesh scene
# scene = trimesh.Scene()
# # Add the object mesh to the scene
# scene.add_geometry(obj_mesh)
# # create a cone to represent the camera
# cone = trimesh.creation.cone(radius=0.01, height=0.05)
# cone.apply_transform(camera_pose)
# scene.add_geometry(cone)
# # Display the scene
# scene.show()



camera = pyrender.camera.IntrinsicsCamera(fx=800, fy=800, cx=320, cy=240)
# Create scene
scene = pyrender.Scene()
# Add mesh to the scene
mesh_node = pyrender.Node(mesh=pyrender.Mesh.from_trimesh(obj_mesh, smooth=False))
# add texture map
scene.add_node(mesh_node)

# Add camera to the scene
scene.add(camera, pose=camera_pose)

# Add light
direct_l = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=1.0)

# Set up the renderer
renderer = pyrender.OffscreenRenderer(viewport_width=640, viewport_height=480)

# Render the camera image
color, depth = renderer.render(scene)

# Display or save the rendered image
# For example, you can use matplotlib to display the image
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(color)
axes[0].set_title('Rendered image')
axes[1].imshow(depth, cmap='gray')
axes[1].set_title('Depth image')
plt.show()
